## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBBBSSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1416
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
|            coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|           res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
|-175.79969703257746|103.65031302067024|-3.9926234135812018|    125.0|BBBBSSBB| 43.1167| -75.2333|       TOBS|0.9598597136618109|0.9527727029356121|0.9527561847960517|0.15576199585176023|USC

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00308733,1938.0,103.650313
1,USC00304102,1932.0,-69.916944
2,USC00303889,1974.0,-5.818612
3,USC00304912,1974.0,-63.557429
4,USC00300785,1963.0,67.358475


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00300379,USC00300424,USC00300668,USC00300785,USC00300870,USC00301110,USC00301708,USC00301732,USC00302047,USC00303087,...,USC00308080,USC00308104,USC00308248,USC00308388,USC00308503,USC00308578,USC00308733,USC00309000,USC00309389,USW00014717
year,,,,,,,,,,,,,,,,,,,,,
1901.0,-64.180984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-131.362699,NaN,NaN,-95.400748,NaN,NaN
1902.0,219.960994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,200.732716,NaN,NaN,375.747729,NaN,235.372903
1903.0,115.781162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,157.021234,NaN,NaN,135.339514,NaN,3.160407
1904.0,-223.205196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-54.901081,NaN,NaN,-66.960989,NaN,107.465103
1905.0,46.526742,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.862282,NaN,NaN,14.132303,NaN,NaN
1906.0,106.955073,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.553109,NaN,NaN
1907.0,121.941076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,74.570574,NaN,111.816265,110.055353,NaN,NaN,176.554282,NaN,NaN
1908.0,-38.690823,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-72.808130,NaN,NaN,-134.895314,NaN,NaN,-145.494936,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,194.572805,NaN,NaN,29.410728,NaN,NaN,104.555412,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  163.457732464
RMS removing mean-by-station=  152.71522681
RMS removing mean-by-year   =  75.7351603322


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 163.457732464
0 after removing mean by year    = 75.7351603322
0 after removing mean by stations= 63.9180986524
1 after removing mean by year    = 63.468494744
1 after removing mean by stations= 63.4047272589
2 after removing mean by year    = 63.3881989128
2 after removing mean by stations= 63.3819380157
3 after removing mean by year    = 63.3790129
3 after removing mean by stations= 63.3775003795
4 after removing mean by year    = 63.3766799238
4 after removing mean by stations= 63.3762244095
